<a href="https://colab.research.google.com/github/Saumyak2682/html-portfolio/blob/main/Final_copy_of_Myntra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests

def fetch_social_media_data(api_url, params):
    response = requests.get(api_url, params=params)
    return response.json()

api_url = "https://api.twitter.com/2/tweets/search/recent"
params = {
    "query": "#fashion",
    "tweet.fields": "created_at,author_id",
    "max_results": 100
}
data = fetch_social_media_data(api_url, params)


In [6]:
import pandas as pd

customer_data = pd.read_csv('customer_data.csv')


In [7]:
survey_data = pd.read_csv('survey_data.csv')


In [8]:
def clean_data(df):
    df.dropna(inplace=True)
    # Additional cleaning steps
    return df

customer_data = clean_data(customer_data)
survey_data = clean_data(survey_data)


In [9]:
from sklearn.decomposition import NMF
from sklearn.impute import SimpleImputer

user_item_matrix = customer_data.pivot(index='user_id', columns='item_id', values='purchase_count')

# Impute missing values with 0 (or any other suitable strategy)
imputer = SimpleImputer(strategy='constant', fill_value=0)
user_item_matrix_imputed = imputer.fit_transform(user_item_matrix)

model = NMF(n_components=10)
user_features = model.fit_transform(user_item_matrix_imputed)  # Use the imputed matrix
item_features = model.components_


In [10]:
import pandas as pd

customer_data = pd.read_csv('customer_data.csv')
survey_data = pd.read_csv('survey_data.csv')

def clean_data(df):
    # Drop rows with missing values in all columns
    df.dropna(how='all', inplace=True)
    # Additional cleaning steps
    return df

customer_data = clean_data(customer_data)
survey_data = clean_data(survey_data)

from sklearn.decomposition import NMF
from sklearn.impute import SimpleImputer

user_item_matrix = customer_data.pivot(index='user_id', columns='item_id', values='purchase_count')

# Impute missing values with 0
imputer = SimpleImputer(strategy='constant', fill_value=0)
user_item_matrix_imputed = imputer.fit_transform(user_item_matrix)

model = NMF(n_components=10)
user_features = model.fit_transform(user_item_matrix_imputed)
item_features = model.components_

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
# Check if 'item_description' column exists before proceeding
if 'item_description' in customer_data.columns:
    item_features = tfidf.fit_transform(customer_data['item_description'])
else:
    print("Warning: 'item_description' column not found in customer_data.")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten

# Initialize the ImageDataGenerator with rescaling
datagen = ImageDataGenerator(rescale=1./255)

from google.colab import drive
drive.mount('/content/drive')

# Update this path to point to your actual training data directory
# Make sure this path is correct and the directory contains images
train_directory = '/content/drive/My Drive/train'

# Create the training data generator
train_generator = datagen.flow_from_directory(train_directory,
                                              target_size=(150, 150),
                                              batch_size=20,
                                              class_mode='binary')

# Verify if the generator found any images
if train_generator.samples == 0:
    print("Error: No images found in the training directory. Please check the path.")
else:
    # Load the VGG16 base model with pre-trained ImageNet weights, excluding the top fully connected layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

    # Create a Sequential model and add the base model and custom layers
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with optimizer, loss function, and metrics
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model using the training data generator
    model.fit(train_generator, steps_per_epoch=1, epochs=50)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 10 images belonging to 5 classes.
58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/50
1/1 [==============================] - 14s 14s/step - loss: 0.7864 - accuracy: 0.3000
Epoch 2/50
1/1 [==============================] - 8s 8s/step - loss: -8.5421 - accuracy: 0.2000
Epoch 3/50
1/1 [==============================] - 11s 11s/step - loss: -8178.6382 - accuracy: 0.2000
Epoch 4/50
1/1 [==============================] - 9s 9s/step - loss: -501787.5625 - accuracy: 0.2000
Epoch 5/50
1/1 [==============================] - 10s 10s/step - loss: -10012078.0000 - accuracy: 0.2000
Epoch 6/50
1/1 [==============================] - 11s 11s/step - loss: -119757992.0000 - accuracy: 0.2000
Epoch 7/50
1/1 [==============================] - 8s 8s/step - loss: -1020523136.0000 - accuracy: 0.2000
Epoch 8/50
1/1 [==============================] - 10s 1

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU

def build_generator():
    model = tf.keras.Sequential()
    model.add(Dense(256, input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    model.add(Reshape((28, 28, 1)))
    return model

def build_discriminator():
    model = tf.keras.Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

def build_gan(generator, discriminator):
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Training the GAN
import numpy as np

def train_gan(gan, generator, discriminator, epochs, batch_size):
    for epoch in range(epochs):
        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_images = generator.predict(noise)
        real_images = np.random.random((batch_size, 28, 28, 1))

        labels_real = np.ones((batch_size, 1))
        labels_fake = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_images, labels_real)
        d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.array([1] * batch_size)
        g_loss = gan.train_on_batch(noise, valid_y)

        print(f"Epoch: {epoch} [D loss: {d_loss[0]}] [G loss: {g_loss}]")

train_gan(gan, generator, discriminator, epochs=10000, batch_size=32)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 24ms/step
Epoch: 128 [D loss: 0.0006861508445581421] [G loss: 7.9850873947143555]
1/1 [==============================] - 0s 25ms/step
Epoch: 129 [D loss: 0.0004884327936451882] [G loss: 8.46699047088623]
1/1 [==============================] - 0s 24ms/step
Epoch: 130 [D loss: 0.0005302098870743066] [G loss: 8.312801361083984]
1/1 [==============================] - 0s 25ms/step
Epoch: 131 [D loss: 0.0005361433431971818] [G loss: 8.39727783203125]
1/1 [==============================] - 0s 26ms/step
Epoch: 132 [D loss: 0.0005063870630692691] [G loss: 8.609996795654297]
1/1 [==============================] - 0s 25ms/step
Epoch: 133 [D loss: 0.0004931466392008588] [G loss: 8.724138259887695]
1/1 [==============================] - 0s 26ms/step
Epoch: 134 [D loss: 0.0004900697094853967] [G loss: 8.526355743408203]
1/1 [==============================] - 0s 24ms/step
Epoch: 135 [D loss: 0.000614321557804

In [ ]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.impute import SimpleImputer
import numpy as np

# Assuming 'customer_data.csv' exists and contains relevant data
customer_data = pd.read_csv('customer_data.csv')

# ... (Rest of your data loading and cleaning code) ...

user_item_matrix = customer_data.pivot(index='user_id', columns='item_id', values='purchase_count')

# Impute missing values with 0
imputer = SimpleImputer(strategy='constant', fill_value=0)
user_item_matrix_imputed = imputer.fit_transform(user_item_matrix)

model = NMF(n_components=10)
user_features = model.fit_transform(user_item_matrix_imputed)  # Assign the result to user_features
item_features = model.components_

# ... (Rest of your code, including hybrid_recommendations function) ...

recommendations = hybrid_recommendations(user_id=1, user_features=user_features, item_features=item_features)
print(recommendations)

In [ ]:
!pip install requests Pillow

import requests
from PIL import Image
import io
import pandas as pd
from IPython.display import display # Used to display images

def fetch_pinterest_data(api_url, params, headers):
    response = requests.get(api_url, params=params, headers=headers)
    return response.json()

# Note: Replace with a valid Pinterest API token for actual data fetching
api_url = "https://api.pinterest.com/v1/me/pins/"
params = {"fields": "id,link,note,creator,board,created_at"}
headers = {"Authorization": "Bearer YOUR_ACCESS_TOKEN"}
# pinterest_data = fetch_pinterest_data(api_url, params, headers)

# Sample pinterest data
pinterest_data = pd.DataFrame({
    'id': [301, 302, 303, 304, 305],
    'link': ['https://i.pinimg.com/564x/f8/05/92/f80592cbd4dbfa3185b6899d2aa79889.jpg',
             'https://i.pinimg.com/736x/9a/80/b9/9a80b99bfbbf15cbf2fdfe186696a315.jpg',
             'https://i.pinimg.com/564x/6b/ea/e2/6beae29e16d27cdeebd111ea2def87c5.jpg',
             'https://i.pinimg.com/564x/6f/8c/14/6f8c14fb4a9a535153f5db945ab64968.jpg',
             'https://i.pinimg.com/564x/27/b6/91/27b69120f62154b3beebc1aa9b2cf9b3.jpg'], # Replace with actual image URLs
    'note': ['Style 1', 'Style 2', 'Style 3', 'Style 4', 'Style 5'],
    'creator': ['User1', 'User2', 'User3', 'User4', 'User5'],
    'board': ['Board1', 'Board2', 'Board3', 'Board4', 'Board5'],
    'created_at': ['2023-06-15', '2023-06-16', '2023-06-17', '2023-06-18', '2023-06-19']
})

def process_image(url):
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content))
    img = img.resize((150, 150))
    return img

# Example of integrating Pinterest data with existing models
def integrate_pinterest_data(pinterest_data, existing_data):
    combined_data = existing_data.append(pinterest_data, ignore_index=True)
    return combined_data

# combined_data = integrate_pinterest_data(pinterest_data, existing_data)

def recommend_similar_products(pinterest_pin):
    img = process_image(pinterest_pin['link'])
    # Use the hybrid recommendation engine to find similar products
    # Note: This part assumes you have a working hybrid_recommendations function and relevant data
    # recommendations = hybrid_recommendations(user_id=1, user_features=user_features, item_features=item_features)
    # For demonstration, let's just display the image
    display(img)
    return # Replace with actual recommendations

# Example pin
pin = pinterest_data.iloc[0]
recommend_similar_products(pin) # Call the function to process and display the image
# print("Recommendations based on Pinterest pin:", recommendations) # Uncomment when you have actual recommendations